# Bài toán

> **Thu thập dữ liệu báo Việt Nam (VnExpress - Demo)**

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://vnexpress.net/chinh-phu-ban-hanh-nghi-dinh-moi-ve-gia-dat-4763835.html",
        "title": "Chính phủ ban hành nghị định mới về giá đất",
        "description": "Chính phủ hôm nay ban hành Nghị định 71, trong đó quy ...",
        "content": "Nghị định này có hiệu lực khi Luật Đất đai 2024 được thi hành ...",
        "metadata": {
            "cat": "Bất động sản",
            "subcat": "Chính sách",
            "published_date": 1719575647,
            "author": "Anh Tú"
        }
    },
    ```

# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [ ]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 44 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package apparmor.
(Reading database ... 123588 files and directories currently installed.)
Preparing to unpack .../apparmor_3.0.4-2ubuntu2.3_amd64.deb ...
Unpacking apparmor (3.0.4-2ubuntu2.3) ...
Selecting previously unselected package liblzo2-2:amd64.
Preparing to unpack .../liblzo2-2_2.10-2build3_amd64.deb ...
Unpack

In [2]:
# Install selenium
%pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir data

Note: you may need to restart the kernel to use updated packages.


A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file data already exists.
Error occurred while processing: data.


In [3]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [81]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [82]:
MAGAZINE_NAME = "vnexpress"
HOME_PAGE = "https://vnexpress.net/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


Vào trang chủ

In [83]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

Chọn menu buttons

In [8]:
# Chọn menu buttons

## Chọn element theo class_name == "all-menu"
all_menu = driver.find_element(by=By.CLASS_NAME, value="all-menu")

## Click vào menu
all_menu.click() #

Thu thập hết thể loại:

* Cách chọn elements ở web trong selenium: https://selenium-python.readthedocs.io/locating-elements.html

In [9]:
# Lấy hết tất cả thể loại từ đây
cats = []

# Chọn row-menu bằng class_name
row_menu = driver.find_element(by=By.CLASS_NAME, value="row-menu")

# Ta thấy mỗi thể loại ở trong element là cat-menu element --> ta lấy hết elements bằng dùng find_elements; find_element -> chọn element đầu tiên match
cat_menus = row_menu.find_elements(by=By.CLASS_NAME, value="cat-menu")

for cat_menu in cat_menus: # Loop through cat menus to main cat and corresponding links
    # Lấy element đầu tiên
    cat = cat_menu.find_element(by=By.TAG_NAME, value="a").get_attribute("title").strip()
    href = cat_menu.find_element(by=By.TAG_NAME, value="a").get_attribute("href").strip()
    # Lưu vào cats list bao gồm tên và url
    cats.append({"cat_name": cat, "url": href})


In [10]:
cats, len(cats)

([{'cat_name': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'},
  {'cat_name': 'Góc nhìn', 'url': 'https://vnexpress.net/goc-nhin'},
  {'cat_name': 'Thế giới', 'url': 'https://vnexpress.net/the-gioi'},
  {'cat_name': 'Video', 'url': 'https://video.vnexpress.net/'},
  {'cat_name': 'Podcasts', 'url': 'https://vnexpress.net/podcast'},
  {'cat_name': 'Kinh doanh', 'url': 'https://vnexpress.net/kinh-doanh'},
  {'cat_name': 'Bất động sản', 'url': 'https://vnexpress.net/bat-dong-san'},
  {'cat_name': 'Khoa học', 'url': 'https://vnexpress.net/khoa-hoc'},
  {'cat_name': 'Giải trí', 'url': 'https://vnexpress.net/giai-tri'},
  {'cat_name': 'Thể thao', 'url': 'https://vnexpress.net/the-thao'},
  {'cat_name': 'Pháp luật', 'url': 'https://vnexpress.net/phap-luat'},
  {'cat_name': 'Giáo dục', 'url': 'https://vnexpress.net/giao-duc'},
  {'cat_name': 'Sức khỏe', 'url': 'https://vnexpress.net/suc-khoe'},
  {'cat_name': 'Đời sống', 'url': 'https://vnexpress.net/doi-song'},
  {'cat_name': 'Du lịch', 'u

In [11]:
# Đóng driver sau khi dùng xong
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt thông số

In [84]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 400 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/vnexpress_url.json"

# Một số thể loại không quan tâm
EXCLUDING_CATEGORIES = ["Video", "Podcasts", "Góc nhìn", "Tâm sự", "Thư giãn", "Ý kiến"]

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"

#### Chạy thử nghiệm

In [85]:
driver = webdriver.Chrome(options=chrome_options)

In [86]:
# Vào thử một url
driver.get(cats[0]['url'])

In [87]:
# Get all item-news class item
title_news = driver.find_elements(by=By.CLASS_NAME, value="title-news")

In [88]:
# Lấy đường dẫn
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
url_new

'https://vnexpress.net/trung-uong-cho-thoi-chuc-mot-bi-thu-3-uy-vien-4776985.html'

In [89]:
# Chỉ lấy links bắt đầu giống HOME_PAGE
url_new.startswith(HOME_PAGE)

True

In [90]:
# Tìm next page
driver.find_element(by=By.CLASS_NAME, value="next-page").get_attribute("href")

'https://vnexpress.net/thoi-su-p2'

In [91]:
driver.close()

#### Chạy thật


In [92]:
driver = webdriver.Chrome(options=chrome_options)

In [94]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    all_urls = set()
    url = category_url

    # Limit the number of NUM_ARTICLES_PER_CAT

    while len(all_urls) < NUM_ARTICLES_PER_CAT:
        driver.get(url)
        title_news = driver.find_elements(by=By.CLASS_NAME, value="title-news")
        for title in title_news:
            try:
                url_new = title.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls: #avoid ads, different sites news
                    all_urls.add(url_new)
                    crawled_urls.add(url_new) # avoid dedup url

            # To see if there is bug
            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                print(f"NoSuchElementException at {url}")
                continue

        url = driver.find_element(by=By.CLASS_NAME, value="next-page").get_attribute("href")

    return all_urls

In [95]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat["cat_name"]
    url = cat["url"]
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

You are at {'cat_name': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'}.
You are at {'cat_name': 'Thế giới', 'url': 'https://vnexpress.net/the-gioi'}.
NoSuchElementException at https://vnexpress.net/the-gioi-p2
NoSuchElementException at https://vnexpress.net/the-gioi-p2
NoSuchElementException at https://vnexpress.net/the-gioi-p2
NoSuchElementException at https://vnexpress.net/the-gioi-p2
NoSuchElementException at https://vnexpress.net/the-gioi-p3
NoSuchElementException at https://vnexpress.net/the-gioi-p3
NoSuchElementException at https://vnexpress.net/the-gioi-p3
NoSuchElementException at https://vnexpress.net/the-gioi-p3
NoSuchElementException at https://vnexpress.net/the-gioi-p4
NoSuchElementException at https://vnexpress.net/the-gioi-p4
NoSuchElementException at https://vnexpress.net/the-gioi-p4
NoSuchElementException at https://vnexpress.net/the-gioi-p4
NoSuchElementException at https://vnexpress.net/the-gioi-p5
NoSuchElementException at https://vnexpress.net/the-gioi-p5
NoSuch

In [96]:
with open(DATA_URL_FILE, "w", encoding='utf-8') as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

In [97]:
len(crawled_urls)

5827

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [98]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vnexpress_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = None # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data/vnexpress"
#
!mkdir -p "$DATA_FOLDER_OUTPUT"

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

A subdirectory or file data/vnexpress already exists.
Error occurred while processing: data/vnexpress.


In [99]:
# Đọc file url
with open(FILE_URL_PATH, "r", encoding='utf-8') as fIn:
    url_data = json.load(fIn)

len(url_data)

14

#### Chạy thử nghiệm

In [49]:
# Một số url để thử nghiệm
SAMPLE_ARTICLE_URLS = [
    "https://vnexpress.net/lao-tuyen-bo-quoc-tang-tuong-niem-tong-bi-thu-nguyen-phu-trong-4772895.html",
    "https://vnexpress.net/ty-le-hoc-sinh-do-tot-nghiep-cao-nhat-10-nam-4772931.html",
    "https://vnexpress.net/tra-luong-cho-nguoi-gia-4772632.html",
    "https://vnexpress.net/cuoc-song-do-thi-mien-nam-hon-100-nam-truoc-4772382.html",
    "https://vnexpress.net/piastri-lan-dau-thang-chang-f1-4772734.html", # table
    "https://vnexpress.net/10-ung-vien-gianh-qua-bong-vang-2024-4772973.html", # slideshow,
    "https://vnexpress.net/toi-kho-chiu-khi-nhieu-nguoi-dung-xe-khi-con-ba-giay-den-xanh-4772885.html", # author,
    "https://vnexpress.net/khong-vo-gao-truoc-khi-nau-co-an-toan-4772624.html",
    "https://vnexpress.net/do-ban-thay-con-cong-khac-biet-trong-10-giay-4770515.html",
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[-1]


In [69]:
SAMPLE_ARTICLE_URL = "https://vnexpress.net/tong-bi-thu-to-lam-cuoc-chien-chong-tham-nhung-se-khong-ngung-nghi-4777266.html"
SAMPLE_ARTICLE_URL = "https://vnexpress.net/phat-hien-gene-chu-dao-ho-tro-dieu-tri-benh-alzheimer-4768256.html"

In [71]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(SAMPLE_ARTICLE_URL)

In [72]:
# Tìm kiếm title
driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text

'Phát hiện gene chủ đạo hỗ trợ điều trị bệnh Alzheimer'

In [73]:
# Tìm kiếm description
driver.find_element(by=By.CLASS_NAME, value="description").text

'MỸTS Võ Văn Giàu, 38 tuổi, cùng cộng sự, phát hiện ra gene chủ đạo ngăn chặn tình trạng chết tế bào não, mở đường cho chẩn đoán và điều trị bệnh Alzheimer.'

In [74]:
# Thu thập thể loại
lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.breadcrumb").find_elements(by=By.TAG_NAME, value="li")
main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None
main_cat, sub_cat

('Khoa học', 'Khoa học trong nước')

In [75]:
# Thu thập ngày
publish_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute("content").strip()
publish_date

'2024-07-26T10:26:22+07:00'

In [76]:
# Tìm kiếm contents
article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
children = article.find_elements(by=By.XPATH, value="./*")

In [77]:
# Thu thập contents và author
contents = []
author = "Unknown"
is_slide_show = False

for idx, child in enumerate(children):
    text = child.text.strip()
    # right align
    if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
        author = text
    elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
        # If center
        if len(text):
            if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
        is_slide_show = True # slideshow
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)

    elif child.tag_name == "table": # Do nothing rightnow
        pass

if is_slide_show:
    author = text

if author == "Unknown":
    try:
        author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
    except:
        pass

In [78]:
contents

["Bệnh suy giảm trí nhớ (Alzheimer's disease - AD) là bệnh lý não bộ gây ra bởi các hiện tượng sinh hóa lý bất thường trong não, dẫn tới chết các tế bào não và làm ảnh hưởng đến khả năng nhận thức của người bệnh. Hiện mới có 6 loại thuốc bao gồm Tacrine, Donepezil, Carbalatine, Galanthamine, Memantine và Lecanemab đã được phê duyệt bởi FDA để sử dụng trong lâm sàng điều trị bệnh Alzheimer. Tuy nhiên thuốc chỉ có thể hỗ trợ cắt giảm triệu chứng chứ chưa loại bỏ tận gốc các tác nhân sinh bệnh cốt lõi liên quan đến sự tích tụ và rối loạn chức năng của hai loại protein chính: beta-amyloid (Aβ) và Tau.",
 'Theo TS Giàu, Viện khám phá Y học Sanford Burnham Prebys (Mỹ), một trong những rào cản cho việc tìm kiếm phương pháp trị liệu hiệu quả ở bệnh Alzheimer là do còn hạn chế hiểu biết về các nguyên nhân sinh học cơ bản bất thường trong quá trình sinh bệnh.',
 'Với vai trò chủ nhiệm dự án, anh cùng nhóm nghiên cứu tại Viện khám phá Y học Sanford Burnham Prebys đã tìm và chứng minh được vai trò

In [79]:
author

'Như Quỳnh'

In [80]:
driver.close()

#### Chạy thật


In [100]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text.strip()

    # Thu thập description
    description = driver.find_element(by=By.CLASS_NAME, value="description").text.strip()

    # Thu thập thể loại
    lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.breadcrumb").find_elements(by=By.TAG_NAME, value="li")
    main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
    sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None

    # Thu thập published date
    publish_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute("content").strip()

    # Thu thập content bài báo
    # Locate phần viết content
    article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")

    contents = []
    author = "Unknown"

    # Check có phải dạng slide show hay không
    is_slide_show = False
    for idx, child in enumerate(children):
        text = child.text.strip()
        # Nếu mà element right align --> có thể là tác giả
        if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
            author = text
        elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
            # If center
            if len(text):
                if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Chỉ lấy caption của figure
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) <= 100: # nếu mà len <= 100 --> add thêm [] xung quanh
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Nếu mà là slide show thì nó giống figure
        elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
            is_slide_show = True # slideshow
            if len(text):
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Bỏ qua table bây giờ
        elif child.tag_name == "table": # Do nothing rightnow
            pass

    if is_slide_show:
        author = text

    # Nếu mà vẫn chưa thấy author thì tìm bằng tag
    if author == "Unknown":
        try:
            author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
        except:
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents), # join các đoạn bằng \n
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [101]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException, NoSuchElementException) as e:
            print(f"Bug at url: {url}, with ElementException")
            driver.refresh()
            # Chúng ta tạm bỏ lỗi bây giờ
            continue

    name_file_cat = cat.lower().replace(" ", "-") + ".json"

    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

Thu thập dữ liệu thể loại Thời sự ..
Bug at url: https://vnexpress.net/di-san-6-thap-ky-cua-tong-bi-thu-nguyen-phu-trong-4773031.html, with ElementException
Thu thập dữ liệu thể loại Thế giới ..
Bug at url: https://vnexpress.net/linh-cuu-hoa-chap-nhan-mat-nha-de-dap-dam-chay-rung-lon-nhat-the-ky-4775972.html, with ElementException
Thu thập dữ liệu thể loại Kinh doanh ..
Bug at url: https://vnexpress.net/topic/gia-vang-lap-dinh-27274, with ElementException
Bug at url: https://vnexpress.net/topic/khung-hoang-egroup-cua-shark-thuy-27423, with ElementException
Bug at url: https://vnexpress.net/daikin-tu-nha-may-linh-kien-den-thuong-hieu-dieu-hoa-ty-usd-4749155.html, with ElementException
Bug at url: https://vnexpress.net/ky-vong-but-pha-tu-cang-hang-khong-quang-tri-4766382.html, with ElementException
Bug at url: https://vnexpress.net/phan-bon-ca-mau-to-chuc-hoi-thao-tai-campuchia-4777333.html, with ElementException
Bug at url: https://vnexpress.net/topic/gia-ve-may-bay-tang-cao-27507, with

In [102]:
# Xem 1 sample
cat_data[0]

{'url': 'https://vnexpress.net/nhung-thuong-hieu-oto-chat-luong-nhat-4765589.html',
 'title': 'Những thương hiệu ôtô chất lượng nhất',
 'description': 'MỸHyundai và Kia đều có tên trong top 5, với thứ hạng tăng vượt bậc so với 2023.',
 'content': 'Nghiên cứu thường niên của J.D. Power - hãng nghiên cứu số liệu thị trường nổi tiếng của Mỹ - dựa trên những rắc rối xảy ra trong quá trình sử dụng ôtô. Số hỏng hóc được tính trên mỗi 100 xe (PP100), và con số này năm nay cao hơn 2023 (195 so với 192).\nDưới đây là 10 thương hiệu ôtô ít xảy ra rắc rối nhất 2024:\nĐứng đầu bảng xếp hạng phân khúc xe phổ thông, cũng là số một về chất lượng của toàn thị trường là Ram - thương hiệu Mỹ thuộc tập đoàn Stellantis. Đứng ngay sau là một hãng Mỹ khác - Chevrolet. Trong xếp hạng 2023, Ram đứng thứ hai, còn Chevrolet thứ 5.\nHyundai và thương hiệu con là Kia lần lượt ở vị trí thứ 3 và 4, với số rắc rối trên mỗi 100 xe chỉ chênh nhau một. Năm ngoái, Kia đứng thứ 9, còn Hyundai là 17.\n[Thương hiệu Kia đứn

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB